In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import pickle
from collections import Counter
import os

notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
csv_path = os.path.join(project_root, "Data", "xp.csv")
csv_path1 = os.path.join(project_root, "Data", "metro.csv")

df = pd.read_csv(csv_path)

metro_df = pd.read_csv(csv_path1)
metro_df.columns = metro_df.columns.str.strip()

In [2]:
class KMeansNumPy:
    """K-Means clustering implementation using only NumPy"""
    
    def __init__(self, n_clusters=8, max_iter=300, n_init=10, random_state=None):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.n_init = n_init
        self.random_state = random_state
        self.cluster_centers_ = None
        self.labels_ = None
        self.inertia_ = None
        
    def _initialize_centroids(self, X, random_state):
        """Initialize centroids using random samples"""
        n_samples = X.shape[0]
        indices = random_state.choice(n_samples, self.n_clusters, replace=False)
        return X[indices].copy()
    
    def _assign_clusters(self, X, centroids):
        """Assign each sample to nearest centroid"""
        distances = np.zeros((X.shape[0], self.n_clusters))
        for i, centroid in enumerate(centroids):
            distances[:, i] = np.sum((X - centroid) ** 2, axis=1)
        labels = np.argmin(distances, axis=1)
        return labels
    
    def _update_centroids(self, X, labels):
        """Update centroids as mean of assigned samples"""
        centroids = np.zeros((self.n_clusters, X.shape[1]))
        for k in range(self.n_clusters):
            mask = labels == k
            if np.sum(mask) > 0:
                centroids[k] = np.mean(X[mask], axis=0)
            else:
                centroids[k] = self.cluster_centers_[k]
        return centroids
    
    def _compute_inertia(self, X, labels, centroids):
        """Compute sum of squared distances to nearest centroid"""
        inertia = 0
        for k in range(self.n_clusters):
            mask = labels == k
            if np.sum(mask) > 0:
                inertia += np.sum((X[mask] - centroids[k]) ** 2)
        return inertia
    
    def fit(self, X):
        """Fit K-Means to data"""
        X = np.asarray(X)
        
        if self.random_state is not None:
            random_state = np.random.RandomState(self.random_state)
        else:
            random_state = np.random.RandomState()
        
        best_inertia = np.inf
        best_centroids = None
        best_labels = None
        
        for _ in range(self.n_init):
            centroids = self._initialize_centroids(X, random_state)
            
            for iteration in range(self.max_iter):
                old_centroids = centroids.copy()
                labels = self._assign_clusters(X, centroids)
                self.cluster_centers_ = centroids
                centroids = self._update_centroids(X, labels)
                
                if np.allclose(centroids, old_centroids):
                    break
            
            inertia = self._compute_inertia(X, labels, centroids)
            
            if inertia < best_inertia:
                best_inertia = inertia
                best_centroids = centroids
                best_labels = labels
        
        self.cluster_centers_ = best_centroids
        self.labels_ = best_labels
        self.inertia_ = best_inertia
        
        return self
    
    def predict(self, X):
        """Predict cluster labels for new data"""
        X = np.asarray(X)
        return self._assign_clusters(X, self.cluster_centers_)
    
    def fit_predict(self, X):
        """Fit and return cluster labels"""
        self.fit(X)
        return self.labels_

In [3]:
class BallTreeNumPy:
    """Ball Tree for efficient nearest neighbor search with haversine distance"""
    
    def __init__(self, data, metric='haversine', leaf_size=40):
        self.data = np.asarray(data)
        self.metric = metric
        self.leaf_size = leaf_size
        self.n_samples = len(data)
        
    def _haversine_distance_vectorized(self, points, reference):
        """Vectorized haversine distance calculation"""
        lat1, lon1 = reference
        lat2 = points[:, 0]
        lon2 = points[:, 1]
        
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        c = 2 * np.arcsin(np.sqrt(np.clip(a, 0, 1)))
        
        return c
    
    def query(self, X, k=1):
        """Find k nearest neighbors for each point in X"""
        X = np.asarray(X)
        n_queries = X.shape[0]
        
        distances = np.zeros((n_queries, k))
        indices = np.zeros((n_queries, k), dtype=int)
        
        for i, query_point in enumerate(X):
            dists = self._haversine_distance_vectorized(self.data, query_point)
            idx = np.argpartition(dists, min(k, len(dists)-1))[:k]
            idx = idx[np.argsort(dists[idx])]
            
            distances[i] = dists[idx]
            indices[i] = idx
        
        return distances, indices
    
    def query_radius(self, X, r):
        """Find all neighbors within radius r for each point in X"""
        X = np.asarray(X)
        results = []
        
        for i, query_point in enumerate(X):
            dists = self._haversine_distance_vectorized(self.data, query_point)
            idx = np.where(dists <= r)[0]
            results.append(idx)
        
        return results

In [4]:
class StandardScalerNumPy:
    """Standardize features by removing mean and scaling to unit variance"""
    
    def __init__(self):
        self.mean_ = None
        self.scale_ = None
        
    def fit(self, X):
        """Compute mean and std for scaling"""
        X = np.asarray(X)
        self.mean_ = np.mean(X, axis=0)
        self.scale_ = np.std(X, axis=0)
        self.scale_[self.scale_ == 0] = 1.0
        return self
    
    def transform(self, X):
        """Scale features"""
        X = np.asarray(X)
        return (X - self.mean_) / self.scale_
    
    def fit_transform(self, X):
        """Fit and transform"""
        return self.fit(X).transform(X)


class SimpleImputerNumPy:
    """Impute missing values"""
    
    def __init__(self, strategy='mean'):
        self.strategy = strategy
        self.statistics_ = None
        
    def fit(self, X):
        """Compute statistics for imputation"""
        X = np.asarray(X)
        
        if self.strategy == 'mean':
            self.statistics_ = np.nanmean(X, axis=0)
        elif self.strategy == 'median':
            self.statistics_ = np.nanmedian(X, axis=0)
        elif self.strategy == 'most_frequent':
            self.statistics_ = np.array([self._most_frequent(X[:, i]) for i in range(X.shape[1])])
        
        return self
    
    def _most_frequent(self, column):
        """Get most frequent value in column"""
        valid = column[~np.isnan(column)]
        if len(valid) == 0:
            return 0
        counter = Counter(valid)
        return counter.most_common(1)[0][0]
    
    def transform(self, X):
        """Impute missing values"""
        X = np.asarray(X, dtype=float).copy()
        
        for i in range(X.shape[1]):
            mask = np.isnan(X[:, i])
            X[mask, i] = self.statistics_[i]
        
        return X
    
    def fit_transform(self, X):
        """Fit and transform"""
        return self.fit(X).transform(X)

In [5]:
class DecisionTreeRegressorNumPy:
    """Fast Decision Tree Regressor using optimized NumPy operations"""
    
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, 
                 max_features=None, random_state=None, criterion='squared_error'):
        self.max_depth = max_depth if max_depth is not None else 999
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.random_state = random_state
        self.criterion = criterion  # 'squared_error' or 'absolute_error'
        self.tree_ = None
        self.feature_importances_ = None
        
    def _compute_error(self, y):
        """Compute error metric for split evaluation"""
        if len(y) == 0:
            return 0
        
        if self.criterion == 'absolute_error':
            median = np.median(y)
            return np.mean(np.abs(y - median))
        else:  # squared_error
            mean = np.mean(y)
            return np.mean((y - mean) ** 2)
    
    def _best_split_fast(self, X, y, feature_indices, indices):
        """Fast best split using vectorized operations and sampling"""
        best_gain = -np.inf
        best_feature = None
        best_threshold = None
        
        current_error = self._compute_error(y[indices])
        n_total = len(indices)
        
        for feature in feature_indices:
            X_feat = X[indices, feature]
            
            # Sample thresholds
            unique_vals = np.unique(X_feat)
            if len(unique_vals) > 20:
                percentiles = np.linspace(5, 95, 20)
                thresholds = np.percentile(X_feat, percentiles)
            else:
                thresholds = unique_vals[:-1]
            
            for threshold in thresholds:
                left_mask = X_feat <= threshold
                n_left = np.sum(left_mask)
                n_right = n_total - n_left
                
                if n_left < self.min_samples_leaf or n_right < self.min_samples_leaf:
                    continue
                
                left_error = self._compute_error(y[indices][left_mask])
                right_error = self._compute_error(y[indices][~left_mask])
                
                weighted_error = (n_left * left_error + n_right * right_error) / n_total
                gain = current_error - weighted_error
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        
        return best_feature, best_threshold
    
    def _build_tree_fast(self, X, y, indices, depth=0):
        """Build tree using indices instead of copying data"""
        n_samples = len(indices)
        
        # Leaf node value depends on criterion
        if self.criterion == 'absolute_error':
            leaf_value = np.median(y[indices])
        else:
            leaf_value = np.mean(y[indices])
        
        # Stopping criteria
        if depth >= self.max_depth or n_samples < self.min_samples_split or np.std(y[indices]) < 1e-7:
            return {'type': 'leaf', 'value': leaf_value, 'n_samples': n_samples}
        
        # Select features
        n_features = X.shape[1]
        if self.max_features == 'sqrt':
            n_select = max(1, int(np.sqrt(n_features)))
            feature_indices = self.rng.choice(n_features, n_select, replace=False)
        elif isinstance(self.max_features, int):
            feature_indices = self.rng.choice(n_features, min(self.max_features, n_features), replace=False)
        else:
            feature_indices = np.arange(n_features)
        
        # Find best split
        feature, threshold = self._best_split_fast(X, y, feature_indices, indices)
        
        if feature is None:
            return {'type': 'leaf', 'value': leaf_value, 'n_samples': n_samples}
        
        # Split indices
        left_mask = X[indices, feature] <= threshold
        left_indices = indices[left_mask]
        right_indices = indices[~left_mask]
        
        # Build subtrees
        return {
            'type': 'node',
            'feature': feature,
            'threshold': threshold,
            'n_samples': n_samples,
            'left': self._build_tree_fast(X, y, left_indices, depth + 1),
            'right': self._build_tree_fast(X, y, right_indices, depth + 1)
        }
    
    def fit(self, X, y):
        """Build decision tree"""
        X = np.asarray(X)
        y = np.asarray(y)
        
        if self.random_state is not None:
            self.rng = np.random.RandomState(self.random_state)
        else:
            self.rng = np.random.RandomState()
        
        self.n_features_ = X.shape[1]
        indices = np.arange(len(X))
        self.tree_ = self._build_tree_fast(X, y, indices)
        
        # Calculate feature importances
        self.feature_importances_ = self._compute_feature_importances()
        
        return self
    
    def _compute_feature_importances(self):
        """Compute feature importances based on tree structure"""
        importances = np.zeros(self.n_features_)
        
        def traverse(node):
            if node['type'] == 'leaf':
                return
            
            feature = node['feature']
            importances[feature] += node['n_samples']
            
            traverse(node['left'])
            traverse(node['right'])
        
        traverse(self.tree_)
        
        if importances.sum() > 0:
            importances = importances / importances.sum()
        
        return importances
    
    def _predict_batch(self, X, tree):
        """Vectorized prediction for batch of samples"""
        n_samples = X.shape[0]
        predictions = np.zeros(n_samples)
        indices = np.arange(n_samples)
        
        def traverse(node, idx):
            if len(idx) == 0:
                return
            
            if node['type'] == 'leaf':
                predictions[idx] = node['value']
                return
            
            left_mask = X[idx, node['feature']] <= node['threshold']
            left_idx = idx[left_mask]
            right_idx = idx[~left_mask]
            
            traverse(node['left'], left_idx)
            traverse(node['right'], right_idx)
        
        traverse(tree, indices)
        return predictions
    
    def predict(self, X):
        """Predict for multiple samples using vectorized approach"""
        X = np.asarray(X)
        return self._predict_batch(X, self.tree_)

In [6]:
class RandomForestRegressorNumPy:
    """Random Forest Regressor using only NumPy"""
    
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2,
                 min_samples_leaf=1, max_features='sqrt', random_state=None, 
                 n_jobs=None, criterion='squared_error'):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.random_state = random_state
        self.n_jobs = n_jobs
        self.criterion = criterion
        self.trees_ = []
        self.feature_importances_ = None
        
    def fit(self, X, y):
        """Build random forest"""
        X = np.asarray(X)
        y = np.asarray(y)
        
        if self.random_state is not None:
            rng = np.random.RandomState(self.random_state)
        else:
            rng = np.random.RandomState()
        
        self.trees_ = []
        n_samples = X.shape[0]
        
        print(f"Training {self.n_estimators} trees...")
        
        for i in range(self.n_estimators):
            if (i + 1) % 10 == 0:
                print(f"  Tree {i + 1}/{self.n_estimators}")
            
            # Bootstrap sampling
            indices = rng.choice(n_samples, n_samples, replace=True)
            X_bootstrap = X[indices]
            y_bootstrap = y[indices]
            
            # Train tree
            tree = DecisionTreeRegressorNumPy(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                max_features=self.max_features,
                criterion=self.criterion,
                random_state=rng.randint(0, 100000)
            )
            tree.fit(X_bootstrap, y_bootstrap)
            self.trees_.append(tree)
        
        # Compute feature importances
        self.feature_importances_ = np.mean([tree.feature_importances_ for tree in self.trees_], axis=0)
        
        print("Training complete!")
        return self
    
    def predict(self, X):
        """Predict by averaging predictions from all trees"""
        X = np.asarray(X)
        predictions = np.array([tree.predict(X) for tree in self.trees_])
        return np.mean(predictions, axis=0)

In [7]:
def mean_absolute_error(y_true, y_pred):
    """Calculate Mean Absolute Error"""
    return np.mean(np.abs(y_true - y_pred))

def mean_squared_error(y_true, y_pred):
    """Calculate Mean Squared Error"""
    return np.mean((y_true - y_pred) ** 2)

def r2_score(y_true, y_pred):
    """Calculate R² Score"""
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)

In [8]:
# Date features
df['date_mutation'] = pd.to_datetime(df['date_mutation'])

df['year'] = df['date_mutation'].dt.year
df['month'] = df['date_mutation'].dt.month
df['day_of_week'] = df['date_mutation'].dt.dayofweek
df['days_since_start'] = (df['date_mutation'] - df['date_mutation'].min()).dt.days

In [9]:
# Geographic clusters using our KMeans implementation
geo_features = df[['longitude', 'latitude']].values
n_clusters = 20

km = KMeansNumPy(n_clusters=n_clusters, random_state=42, n_init=10)
df['geo_cluster'] = km.fit_predict(geo_features)

# Distance from center
center_lon, center_lat = 2.3384444444444446, 48.86152777777778

df['dist_center'] = np.sqrt((df['longitude'] - center_lon)**2 +
                             (df['latitude'] - center_lat)**2) * 111

In [10]:
# Metro station features using our BallTree implementation
metro_coords = np.radians(metro_df[['Latitude', 'Longitude']].values)
tree = BallTreeNumPy(metro_coords, metric='haversine')

appart_coords = np.radians(df[['latitude', 'longitude']].values)

distances, indices = tree.query(appart_coords, k=1)
df['nearest_metro_dist_km'] = distances.flatten() * 6371

# Add nearest metro station info
df['nearest_metro_station'] = metro_df.iloc[indices.flatten()]['Libelle station'].values
df['nearest_metro_line'] = metro_df.iloc[indices.flatten()]['Libelle Line'].values

# Count metro stations within different radii
indices_300m = tree.query_radius(appart_coords, r=0.3/6371)
indices_500m = tree.query_radius(appart_coords, r=0.5/6371)

df['metro_count_300m'] = [len(idx) for idx in indices_300m]
df['metro_count_500m'] = [len(idx) for idx in indices_500m]
df['very_close_to_metro'] = (df['nearest_metro_dist_km'] < 0.1).astype(int)

In [11]:
# Property features
df['surface_per_piece'] = df['surface_reelle_bati'] / df['nombre_pieces_principales'].replace(0, 1)
df['is_studio'] = (df['nombre_pieces_principales'] == 1).astype(int)
df['is_large'] = (df['nombre_pieces_principales'] >= 4).astype(int)

# Surface categories
df['surface_category'] = pd.cut(df['surface_reelle_bati'], 
                                  bins=[9, 40, 80, float('inf')],
                                  labels=['small', 'medium', 'large'])

# IMPORTANT: Add price_per_sqrtm column (the target in sklearn notebook)
df['price_per_sqrtm'] = df['valeur_fonciere'] / df['surface_reelle_bati']

In [12]:
# Sort by date
df_sorted = df.sort_values('date_mutation').reset_index(drop=True)

# Split: 80% train, 20% test
split_index = int(len(df_sorted) * 0.8)

train_df = df_sorted.iloc[:split_index].copy()
test_df = df_sorted.iloc[split_index:].copy()

In [13]:
station_stats_train = train_df.groupby('nearest_metro_station').agg({
    'surface_reelle_bati': ['mean', 'std', 'median', 'count'],
    'nombre_pieces_principales': ['mean', 'std', 'median'],
}).round(2)

# Flatten column names
station_stats_train.columns = [
    'station_avg_surface',
    'station_surface_std',
    'station_median_surface',
    'station_tx_count',
    'station_avg_rooms',
    'station_rooms_std',
    'station_median_rooms'
]

# Add derived features
station_stats_train['station_surface_range'] = (
    station_stats_train['station_avg_surface'] + 2*station_stats_train['station_surface_std'] -
    (station_stats_train['station_avg_surface'] - 2*station_stats_train['station_surface_std'])
)

# Replace zero std with 1 to avoid division by zero
station_stats_train['station_surface_std'] = station_stats_train['station_surface_std'].replace(0, 1)
station_stats_train['station_rooms_std'] = station_stats_train['station_rooms_std'].replace(0, 1)

In [ ]:
# Merge to TRAIN
train_df = train_df.merge(
    station_stats_train,
    left_on='nearest_metro_station',
    right_index=True,
    how='left'
)

# Merge to TEST
test_df = test_df.merge(
    station_stats_train,
    left_on='nearest_metro_station',
    right_index=True,
    how='left'
)

# Handle new stations in test
station_cols = [c for c in train_df.columns if c.startswith('station_')]

for col in station_cols:
    train_mean = train_df[col].mean()
    
    train_missing = train_df[col].isna().sum()
    test_missing = test_df[col].isna().sum()
    
    if train_missing > 0 or test_missing > 0:
        print(f"  {col}: filling {test_missing} missing values in test with train mean {train_mean:.2f}")
    
    train_df[col].fillna(train_mean, inplace=True)
    test_df[col].fillna(train_mean, inplace=True)


Handling missing values for new stations in test set...


C:\Users\souri\AppData\Local\Temp\ipykernel_5112\180845287.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(train_mean, inplace=True)
C:\Users\souri\AppData\Local\Temp\ipykernel_5112\180845287.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

In [15]:
# Derived features based on station stats - MATCHING SKLEARN NOTEBOOK EXACTLY
for dataset_name, dataset in [('TRAIN', train_df), ('TEST', test_df)]:
    dataset['surface_vs_station_avg'] = (
        (dataset['surface_reelle_bati'] - dataset['station_avg_surface']) / 
        dataset['station_surface_std']
    )
    
    dataset['rooms_vs_station_avg'] = (
        (dataset['nombre_pieces_principales'] - dataset['station_avg_rooms']) / 
        dataset['station_rooms_std']
    )
    
    dataset['larger_than_station_median'] = (
        dataset['surface_reelle_bati'] > dataset['station_median_surface']
    ).astype(int)

In [16]:
# Aggregate by (code_postal, year, month) - EXACTLY as in sklearn notebook
train_agg = train_df.groupby(['code_postal', 'year', 'month']).agg({
    'price_per_sqrtm': 'median',
    'valeur_fonciere': 'median',
    'nombre_pieces_principales': 'median',
    'surface_reelle_bati': 'median',
    'surface_per_piece': 'median',
    'longitude': 'mean',
    'latitude': 'mean',
    'days_since_start': 'median',
    'geo_cluster': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],
    'dist_center': 'median',
    'nearest_metro_dist_km': 'median',
    'station_tx_count': 'median',
    'station_avg_surface': 'median',
    'station_surface_std': 'median',
    'station_median_surface': 'median',
    'station_avg_rooms': 'median',
    'station_rooms_std': 'median',
    'station_median_rooms': 'median',
    'station_surface_range': 'median',
    'surface_vs_station_avg': 'median',
    'rooms_vs_station_avg': 'median',
    'larger_than_station_median': 'median',
    'metro_count_300m': 'median',
    'metro_count_500m': 'median',
    'very_close_to_metro': 'median',
    'is_studio': 'median',
    'is_large': 'median',
    'date_mutation': 'first'
}).reset_index()

print(f"Train aggregated shape: {train_agg.shape}")

Train aggregated shape: (837, 31)


In [17]:
test_agg = test_df.groupby(['code_postal', 'year', 'month']).agg({
    'price_per_sqrtm': 'median',
    'valeur_fonciere': 'median',
    'nombre_pieces_principales': 'median',
    'surface_reelle_bati': 'median',
    'surface_per_piece': 'median',
    'longitude': 'mean',
    'latitude': 'mean',
    'days_since_start': 'median',
    'geo_cluster': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],
    'dist_center': 'median',
    'nearest_metro_dist_km': 'median',
    'station_tx_count': 'median',
    'station_avg_surface': 'median',
    'station_surface_std': 'median',
    'station_median_surface': 'median',
    'station_avg_rooms': 'median',
    'station_rooms_std': 'median',
    'station_median_rooms': 'median',
    'station_surface_range': 'median',
    'surface_vs_station_avg': 'median',
    'rooms_vs_station_avg': 'median',
    'larger_than_station_median': 'median',
    'metro_count_300m': 'median',
    'metro_count_500m': 'median',
    'very_close_to_metro': 'median',
    'is_studio': 'median',
    'is_large': 'median',
    'date_mutation': 'first'
}).reset_index()

print(f"Test aggregated shape: {test_agg.shape}")

Test aggregated shape: (260, 31)


In [18]:
# Transaction counts per postal code per month
train_tx_counts = train_df.groupby(['code_postal', 'year', 'month']).size().reset_index(name='transaction_count')
test_tx_counts = test_df.groupby(['code_postal', 'year', 'month']).size().reset_index(name='transaction_count')

train_agg = train_agg.merge(train_tx_counts, on=['code_postal', 'year', 'month'])
test_agg = test_agg.merge(test_tx_counts, on=['code_postal', 'year', 'month'])

# Historical activity - compute ONLY on training data
print("\nComputing historical transaction counts (TRAIN DATA ONLY)...")
historical_activity_train = train_df.groupby('code_postal').size().reset_index(name='total_transactions_train')

# Merge to both train and test (using TRAIN statistics)
train_agg = train_agg.merge(historical_activity_train, on='code_postal', how='left')
test_agg = test_agg.merge(historical_activity_train, on='code_postal', how='left')

# Handle postal codes in test that weren't in train
overall_avg_transactions = historical_activity_train['total_transactions_train'].mean()
test_agg['total_transactions_train'] = test_agg['total_transactions_train'].fillna(overall_avg_transactions)

# Market activity ratio
train_agg['market_activity_ratio'] = train_agg['transaction_count'] / train_agg['total_transactions_train']
test_agg['market_activity_ratio'] = test_agg['transaction_count'] / test_agg['total_transactions_train']


Computing historical transaction counts (TRAIN DATA ONLY)...


## Prepare Data for Modeling

In [19]:
# Define target and features - MATCHING SKLEARN NOTEBOOK
target = 'price_per_sqrtm'
drop_cols = ['price_per_sqrtm', 'valeur_fonciere', 'date_mutation']

# Split features and target
x_train = train_agg.drop(columns=drop_cols)
y_train = train_agg[target]

x_test = test_agg.drop(columns=drop_cols)
y_test = test_agg[target]

print(f"Training set: {x_train.shape}")
print(f"Test set: {x_test.shape}")

Training set: (837, 31)
Test set: (260, 31)


In [20]:
# Identify numeric features only (no categorical in this dataset)
numeric_features = x_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\nNumeric features: {len(numeric_features)}")
print(numeric_features)


Numeric features: 29
['code_postal', 'nombre_pieces_principales', 'surface_reelle_bati', 'surface_per_piece', 'longitude', 'latitude', 'days_since_start', 'geo_cluster', 'dist_center', 'nearest_metro_dist_km', 'station_tx_count', 'station_avg_surface', 'station_surface_std', 'station_median_surface', 'station_avg_rooms', 'station_rooms_std', 'station_median_rooms', 'station_surface_range', 'surface_vs_station_avg', 'rooms_vs_station_avg', 'larger_than_station_median', 'metro_count_300m', 'metro_count_500m', 'very_close_to_metro', 'is_studio', 'is_large', 'transaction_count', 'total_transactions_train', 'market_activity_ratio']


## Preprocessing Pipeline

In [ ]:
# Extract numeric data
X_train_numeric = x_train[numeric_features].values
X_test_numeric = x_test[numeric_features].values

# Impute missing values with median (matching sklearn notebook)
imputer = SimpleImputerNumPy(strategy='median')
X_train_imputed = imputer.fit_transform(X_train_numeric)
X_test_imputed = imputer.transform(X_test_numeric)

# Standardize features
scaler = StandardScalerNumPy()
X_train_processed = scaler.fit_transform(X_train_imputed)
X_test_processed = scaler.transform(X_test_imputed)

print("Preprocessing complete!")
print(f"Processed training shape: {X_train_processed.shape}")
print(f"Processed test shape: {X_test_processed.shape}")

Imputing missing values with median...
Standardizing features...
Preprocessing complete!
Processed training shape: (837, 29)
Processed test shape: (260, 29)


## Train Random Forest Model (Matching sklearn parameters)

In [ ]:
# BASELINE MODEL - matching sklearn baseline
print("TRAINING BASELINE MODEL")

baseline_model = RandomForestRegressorNumPy(
    n_estimators=200,
    max_depth=30,
    min_samples_leaf=1,
    min_samples_split=15,
    random_state=42,
    criterion='absolute_error'  # Matching sklearn
)

baseline_model.fit(X_train_processed, y_train.values)


TRAINING BASELINE MODEL
Training 200 trees...
  Tree 10/200
  Tree 20/200
  Tree 30/200
  Tree 40/200
  Tree 50/200
  Tree 60/200
  Tree 70/200
  Tree 80/200
  Tree 90/200
  Tree 100/200
  Tree 110/200
  Tree 120/200
  Tree 130/200
  Tree 140/200
  Tree 150/200
  Tree 160/200
  Tree 170/200
  Tree 180/200
  Tree 190/200
  Tree 200/200
Training complete!


In [ ]:
# Baseline predictions
y_train_pred = baseline_model.predict(X_train_processed)
y_test_pred = baseline_model.predict(X_test_processed)

# Evaluate baseline
baseline_results = {
    'train_r2': r2_score(y_train.values, y_train_pred),
    'test_r2': r2_score(y_test.values, y_test_pred),
    'train_mae': mean_absolute_error(y_train.values, y_train_pred),
    'test_mae': mean_absolute_error(y_test.values, y_test_pred),
    'train_rmse': np.sqrt(mean_squared_error(y_train.values, y_train_pred)),
    'test_rmse': np.sqrt(mean_squared_error(y_test.values, y_test_pred))
}

print("BASELINE MODEL RESULTS (NumPy Implementation)")
print(f"Train R²:  {baseline_results['train_r2']:.4f}")
print(f"Test R²:   {baseline_results['test_r2']:.4f}")
print(f"\nTrain MAE:  {baseline_results['train_mae']:.2f} €/m²")
print(f"Test MAE:   {baseline_results['test_mae']:.2f} €/m²")
print(f"\nTrain RMSE: {baseline_results['train_rmse']:.2f} €/m²")
print(f"Test RMSE:  {baseline_results['test_rmse']:.2f} €/m²")

print("COMPARISON TO SKLEARN BASELINE")
print("sklearn results:")
print("  Train R²:  0.9645")
print("  Test R²:   0.9083")
print("  Train MAE:  205.32 €/m²")
print("  Test MAE:   353.10 €/m²")
print("  Train RMSE: 302.25 €/m²")
print("  Test RMSE:  508.44 €/m²")



BASELINE MODEL RESULTS (NumPy Implementation)
Train R²:  0.9658
Test R²:   0.8556

Train MAE:  212.96 €/m²
Test MAE:   489.56 €/m²

Train RMSE: 296.68 €/m²
Test RMSE:  637.86 €/m²

COMPARISON TO SKLEARN BASELINE
sklearn results:
  Train R²:  0.9645
  Test R²:   0.9083
  Train MAE:  205.32 €/m²
  Test MAE:   353.10 €/m²
  Train RMSE: 302.25 €/m²
  Test RMSE:  508.44 €/m²


In [ ]:
# OPTIMIZED MODEL - matching sklearn's best parameters from GridSearch
print("TRAINING OPTIMIZED MODEL")
print("Parameters: n_estimators=250, max_depth=25, min_samples_split=25, min_samples_leaf=1")

optimized_model = RandomForestRegressorNumPy(
    n_estimators=250,
    max_depth=25,
    min_samples_leaf=1,
    min_samples_split=25,
    random_state=42,
    criterion='absolute_error'
)

optimized_model.fit(X_train_processed, y_train.values)


TRAINING OPTIMIZED MODEL
Using sklearn's best parameters: n_estimators=250, max_depth=25, min_samples_split=25, min_samples_leaf=1
Training 250 trees...
  Tree 10/250
  Tree 20/250
  Tree 30/250
  Tree 40/250
  Tree 50/250
  Tree 60/250
  Tree 70/250
  Tree 80/250
  Tree 90/250
  Tree 100/250
  Tree 110/250
  Tree 120/250
  Tree 130/250
  Tree 140/250
  Tree 150/250
  Tree 160/250
  Tree 170/250
  Tree 180/250
  Tree 190/250
  Tree 200/250
  Tree 210/250
  Tree 220/250
  Tree 230/250
  Tree 240/250
  Tree 250/250
Training complete!


In [ ]:
# Optimized predictions
y_train_pred_opt = optimized_model.predict(X_train_processed)
y_test_pred_opt = optimized_model.predict(X_test_processed)

# Evaluate optimized
optimized_results = {
    'train_r2': r2_score(y_train.values, y_train_pred_opt),
    'test_r2': r2_score(y_test.values, y_test_pred_opt),
    'train_mae': mean_absolute_error(y_train.values, y_train_pred_opt),
    'test_mae': mean_absolute_error(y_test.values, y_test_pred_opt),
    'train_rmse': np.sqrt(mean_squared_error(y_train.values, y_train_pred_opt)),
    'test_rmse': np.sqrt(mean_squared_error(y_test.values, y_test_pred_opt))
}

print("OPTIMIZED MODEL RESULTS (NumPy Implementation)")
print(f"Train R²:  {optimized_results['train_r2']:.4f}")
print(f"Test R²:   {optimized_results['test_r2']:.4f}")
print(f"\nTrain MAE:  {optimized_results['train_mae']:.2f} €/m²")
print(f"Test MAE:   {optimized_results['test_mae']:.2f} €/m²")
print(f"\nTrain RMSE: {optimized_results['train_rmse']:.2f} €/m²")
print(f"Test RMSE:  {optimized_results['test_rmse']:.2f} €/m²")

print("COMPARISON TO SKLEARN OPTIMIZED")
print("sklearn results:")
print("  Train R²:  0.9493")
print("  Test R²:   0.9009")
print("  Train MAE:  246.54 €/m²")
print("  Test MAE:   375.53 €/m²")
print("  Train RMSE: 361.29 €/m²")
print("  Test RMSE:  528.34 €/m²")



OPTIMIZED MODEL RESULTS (NumPy Implementation)
Train R²:  0.9506
Test R²:   0.8370

Train MAE:  258.47 €/m²
Test MAE:   536.64 €/m²

Train RMSE: 356.58 €/m²
Test RMSE:  677.78 €/m²

COMPARISON TO SKLEARN OPTIMIZED
sklearn results:
  Train R²:  0.9493
  Test R²:   0.9009
  Train MAE:  246.54 €/m²
  Test MAE:   375.53 €/m²
  Train RMSE: 361.29 €/m²
  Test RMSE:  528.34 €/m²


## Feature Importance

In [26]:
# Create feature importance dataframe
feature_importance = pd.DataFrame({
    'feature_names': [f'num__{f}' for f in numeric_features],
    'coefficients': optimized_model.feature_importances_
})

feature_importance_sorted = feature_importance.sort_values('coefficients', ascending=False)

print("\nTop 10 Most Important Features:")
print(feature_importance_sorted.head(10))


Top 10 Most Important Features:
                    feature_names  coefficients
6           num__days_since_start      0.079924
0                num__code_postal      0.074822
12       num__station_surface_std      0.067462
17     num__station_surface_range      0.062694
5                   num__latitude      0.055374
15         num__station_rooms_std      0.050895
10          num__station_tx_count      0.050651
27  num__total_transactions_train      0.050197
26         num__transaction_count      0.049801
4                  num__longitude      0.049553


In [27]:
# Visualize feature importance
fig1 = px.bar(
    feature_importance_sorted,
    x="feature_names",
    y="coefficients",
    title="Random Forest Feature Importance - NumPy Implementation",
    labels={
        'coefficients': 'Importance',
        'feature_names': 'Features'
    }
)

fig1.show()

## Save Model and Preprocessor

In [ ]:
# Create preprocessor object
preprocessor = {
    'imputer': imputer,
    'scaler': scaler,
    'feature_names': numeric_features
}

# Save optimized model
with open("../src/model_numpy.pkl", "wb") as f:
    pickle.dump(optimized_model, f)

# Save preprocessor
with open("../src/preprocessor_numpy.pkl", "wb") as f:
    pickle.dump(preprocessor, f)

print("\nModel saved")



Model and preprocessor saved successfully!
  - model_numpy_optimized.pkl
  - preprocessor_numpy.pkl


## Summary

This notebook reproduces the sklearn pipeline EXACTLY using only NumPy:

**Key Matches:**
- Same target: `price_per_sqrtm`
- Same aggregation: by (code_postal, year, month)
- Same preprocessing: median imputation + standardization
- Same model parameters: Random Forest with absolute_error criterion
- Baseline: 200 trees, depth 30, min_samples_split 15
- Optimized: 250 trees, depth 25, min_samples_split 25

**Custom NumPy Implementations:**
- K-Means clustering
- Ball Tree (haversine distance)
- Standard Scaler
- Simple Imputer (median strategy)
- Decision Tree Regressor (with MAE criterion)
- Random Forest Regressor
- Evaluation metrics (MAE, RMSE, R²)

Now you can directly compare NumPy vs sklearn performance on the SAME problem!